In [43]:
import cv2
import numpy as np

In [44]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [45]:
with_mask.shape

(200, 50, 50, 3)

In [46]:
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [47]:
X=np.r_[with_mask, without_mask]

In [48]:
X.shape

(400, 7500)

In [49]:
labels = np.zeros(X.shape[0])

In [50]:
labels[200:]= 1.0

In [51]:
names = {0: 'Mask', 1: 'No Mask'}

In [52]:
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.20)

In [55]:
x_train.shape

(320, 7500)

In [56]:
from sklearn.decomposition import PCA

In [57]:
pca = PCA(n_components=3)
x_train= pca.fit_transform(x_train)

In [58]:
x_train.shape

(320, 3)

In [59]:
x_train[0]

array([   73.4980275 , -1348.85094262,   421.59606951])

In [60]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [61]:
x_test = pca.fit_transform(x_test)
y_pred = svm.predict(x_test)

In [62]:
accuracy_score(y_test, y_pred)

0.9

In [63]:
haar_data = cv2.CascadeClassifier('data.xml')
capture = cv2.VideoCapture(0)
data = []
font= cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 255), 4)
            face= img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50, 50))
            face = face.reshape(1, -1)
            face= pca.transform(face)
            pred= svm.predict(face)[0]
            n=names[int(pred)]
            cv2.putText(img, n, (x, y), font, 1, (244, 255, 250), 2)
#             print(n)
        cv2.imshow('result', img)
        if cv2.waitKey(2)==27:
            break
capture.release()
cv2.destroyAllWindows()